In [1]:
import globs
import grid
import data_processing
import gmaps
from math import isnan


In [17]:
#data need preparation first
def make_maps(load_file,border,key,verbose=False):
    squares_basic_data, squares_with_data = data_processing.load_data_from_file(load_file, verbose=verbose)
    gmaps.configure(api_key=key)
    lat_c, lon_c = grid.calculate_center_point(border)

    locations = []
    stds = []
    means = []
    mins = []
    maxs = []
    for pol, mean, std, min2, max2 in squares_basic_data:
        locations.append(grid.calculate_center_point(pol))
        stds.append(std)
        means.append(mean)
        mins.append(min2)
        maxs.append(max2)
        
    figs = []
    for fig_type,data in [("Min", mins), ("Max", maxs), ("Mean", means), ("Std", stds)]:
        fig = gmaps.figure(center=(lat_c, lon_c), zoom_level=9)
#         print(data)
        heat_layer = gmaps.heatmap_layer(locations=locations, weights=data,point_radius=35)
        fig.add_layer(heat_layer)
        figs.append(fig)
    return figs

    
    


In [18]:
file = globs.cities.TOKYO_FILE
border = globs.cities.TOKYO
key = ""

figs = make_maps(file,border,key,False)

In [19]:
#MIN
figs[0]

Figure(layout=FigureLayout(height='420px'))

In [20]:
#MAX
figs[1]

Figure(layout=FigureLayout(height='420px'))

In [21]:
#MEAN
figs[2]

Figure(layout=FigureLayout(height='420px'))

In [22]:
#STD
figs[3]

Figure(layout=FigureLayout(height='420px'))